# CrewAI Demo

This notebook demonstrates **CrewAI**, a comprehensive framework for orchestrating role-based multi-agent teams that mimic real-world human crew dynamics.

## Required Environment Variables

```
OPENAI_API_KEY    # Your OpenAI API key (required)
SERPER_API_KEY    # Your Serper API key (optional, for web search)
```

Please refer to the [README](README.md) for instructions on setting up environment variables.


In [ ]:
# If running standalone without the project setup:
# pip install crewai crewai-tools openai python-dotenv

In [ ]:
import os
import yaml
from pathlib import Path
from dotenv import load_dotenv
from typing import Any, Type, Optional
from pydantic import BaseModel, Field

# Load environment variables
load_dotenv(override=True)

# Import CrewAI components
from crewai import Agent, Task, Crew, Process
from crewai_tools import SerperDevTool
from crewai.tools import BaseTool

### CrewAI Core Components Loaded:
- **Agent:** Role-based AI team members
- **Task:** Structured work items
- **Crew:** Team orchestration
- **Process:** Execution workflows

## 🎯 What We're Building

In this demo, we'll create a **Content Marketing Team** using CrewAI to demonstrate how AI agents can work together like a real marketing department.

### The Scenario
We're building an automated workflow to create a blog post about "AI-powered content marketing" - a task that typically requires:
- **Research** → gathering market insights and trends
- **Writing** → transforming research into engaging content  
- **Editing** → polishing for quality and SEO


## 🛠️ Tool Definitions

Before defining our agents, we need to set up the tools they'll use. CrewAI supports both built-in tools and custom implementations.

### Tool Categories:
1. **Research Tools** - Web search for gathering information
2. **Content Tools** - Writing assistance, editing, and SEO optimization

Let's define the essential tools our agents will need:

In [ ]:
# Define input schemas for tools - flexible to work with CrewAI
from typing import Optional

class WritingAssistantInput(BaseModel):
    """Input schema for WritingAssistantTool."""
    prompt: Optional[str] = Field(None, description="Writing task or prompt")
    description: Optional[str] = Field(None, description="Task description from CrewAI")

    @property
    def get_input(self) -> str:
        """Get the actual input string from either field."""
        return self.description or self.prompt or ""

class EditingAssistantInput(BaseModel):
    """Input schema for EditingAssistantTool."""
    content: Optional[str] = Field(None, description="Content to edit")
    description: Optional[str] = Field(None, description="Task description from CrewAI")

    @property
    def get_input(self) -> str:
        """Get the actual input string from either field."""
        return self.description or self.content or ""

class SEOOptimizerInput(BaseModel):
    """Input schema for SEOOptimizerTool."""
    content: Optional[str] = Field(None, description="Content to optimize for SEO")
    description: Optional[str] = Field(None, description="Task description from CrewAI")

    @property
    def get_input(self) -> str:
        """Get the actual input string from either field."""
        return self.description or self.content or ""

# Define custom tool classes for content creation workflow
class WritingAssistantTool(BaseTool):
    """Tool to assist with content writing and generation."""
    name: str = "Writing Assistant"
    description: str = "Helps generate, structure, and improve written content"
    args_schema: Type[BaseModel] = WritingAssistantInput

    def _run(self, **kwargs) -> str:
        """Assist with writing tasks."""
        # Create input object and get the actual input
        input_obj = WritingAssistantInput(**kwargs)
        prompt = input_obj.get_input

        return f"""Writing Assistant Suggestions:
        - Start with a compelling hook
        - Use clear subheadings for structure
        - Include statistics and data points
        - Add real-world examples
        - End with actionable takeaways
        - Keep paragraphs concise (3-4 sentences)
        - Use active voice for engagement"""

class EditingAssistantTool(BaseTool):
    """Tool for editorial assistance."""
    name: str = "Editing Assistant"
    description: str = "Provides editorial suggestions for content improvement"
    args_schema: Type[BaseModel] = EditingAssistantInput

    def _run(self, **kwargs) -> str:
        """Provide editorial suggestions."""
        # Create input object and get the actual input
        input_obj = EditingAssistantInput(**kwargs)
        content = input_obj.get_input

        return """Editorial Suggestions:
        - Strengthen the introduction with a compelling statistic
        - Break up long paragraphs for better readability
        - Add transition phrases between sections
        - Ensure consistent tone throughout
        - Verify all facts and statistics
        - Include a clear call-to-action at the end"""

class SEOOptimizerTool(BaseTool):
    """Tool for SEO optimization."""
    name: str = "SEO Optimizer"
    description: str = "Optimizes content for search engines"
    args_schema: Type[BaseModel] = SEOOptimizerInput

    def _run(self, **kwargs) -> str:
        """Optimize for SEO."""
        # Create input object and get the actual input
        input_obj = SEOOptimizerInput(**kwargs)
        content = input_obj.get_input

        return """SEO Optimization Suggestions:
        - Primary keyword: 'AI content marketing' (use 3-5 times)
        - Include long-tail keywords naturally
        - Meta description: 150-160 characters
        - Use H2 and H3 tags for structure
        - Add internal and external links
        - Optimize images with alt text
        - Target 1200-1500 words for depth"""

# Initialize all tools
print("Initializing CrewAI Tools...")
print("=" * 50)

# Research Tools
try:
    web_search_tool = SerperDevTool() if os.getenv('SERPER_API_KEY') else None
    if web_search_tool:
        print("✓ Web Search Tool (SerperDev) - Initialized")
    else:
        print("✗ Web Search Tool - No SERPER_API_KEY found (will use mock data)")
except Exception as e:
    web_search_tool = None
    print(f"✗ Web Search Tool - Error: {e}")

# Content Tools
writing_assistant_tool = WritingAssistantTool()
print("✓ Writing Assistant Tool - Initialized")

editing_assistant_tool = EditingAssistantTool()
print("✓ Editing Assistant Tool - Initialized")

seo_optimizer_tool = SEOOptimizerTool()
print("✓ SEO Optimizer Tool - Initialized")

print("=" * 50)
print(f"Total tools initialized: 4")

# Create tool mapping for easy reference
tools_registry = {
    'web_search': web_search_tool,
    'writing_assistant': writing_assistant_tool,
    'editing_assistant': editing_assistant_tool,
    'seo_optimizer': seo_optimizer_tool
}

# Display tool availability
available_tools = [name for name, tool in tools_registry.items() if tool is not None]
print(f"Available tools: {', '.join(available_tools)}")

### Tool Availability and Fallback Behavior

When tools are not available (e.g., missing API keys), CrewAI handles this gracefully:

1. **Explicit Tools**: Custom tools we define are always available
2. **API-based Tools**: May fail if API keys are missing (e.g., SerperDevTool)
3. **LLM Fallback**: Agents can use their LLM capabilities when tools are unavailable
4. **Tool Simulation**: CrewAI can simulate tool behavior using the LLM

This design ensures the demo works even without all API keys configured.

## 🤖 Agent Definition

CrewAI agents are **role-based team members** with comprehensive personas. Each agent has:

### Core Agent Attributes:
- **Role** - The agent's job title/specialization
- **Goal** - What the agent aims to achieve
- **Backstory** - Context and expertise background
- **Tools** - Available capabilities and integrations
- **LLM** - The underlying language model


In [ ]:
# Create agents.yaml configuration - Shows how tools map to agents
agents_yaml_content = """
content_researcher:
  role: Senior Content Researcher
  goal: Research and gather comprehensive information on given topics to support content creation
  backstory: You are an experienced researcher with 10+ years in digital marketing research. You excel at finding credible sources, identifying market trends, and extracting actionable insights from complex data.
  tools:
  - web_search         # SerperDevTool (if API key available)
  verbose: true
  allow_delegation: false

content_writer:
  role: Senior Content Writer
  goal: Create engaging, high-quality content based on research and brand guidelines
  backstory: You are a seasoned content writer with expertise in creating compelling marketing content. You understand how to transform research into engaging narratives that resonate with target audiences.
  tools:
  - writing_assistant  # WritingAssistantTool (custom)
  verbose: true
  allow_delegation: false

content_editor:
  role: Senior Content Editor
  goal: Review, refine, and ensure content quality meets brand standards and objectives
  backstory: You are an experienced content editor with a keen eye for detail, brand consistency, and audience engagement. You excel at polishing content to perfection while maintaining the original message.
  tools:
  - editing_assistant  # EditingAssistantTool (custom)
  - seo_optimizer      # SEOOptimizerTool (custom)
  verbose: true
  allow_delegation: false
"""

# Parse YAML for documentation purposes
agents_config = yaml.safe_load(agents_yaml_content.strip())

print("Agent Configuration (YAML):")
print("=" * 50)
for agent_name, config in agents_config.items():
    print(f"\n{agent_name}:")
    print(f"  Role: {config['role']}")
    print(f"  Tools: {', '.join(config['tools'])}")
    print(f"  Delegation: {'Enabled' if config.get('allow_delegation', False) else 'Disabled'}")
print("=" * 50)
print("\nNote: This YAML shows the ideal configuration. Actual tool availability")
print("depends on API keys and successful initialization (see tool setup above).")

In [ ]:
# Create Agent instances with properly defined tools

# Build tool lists for each agent
researcher_tools = []
if web_search_tool:
    researcher_tools.append(web_search_tool)

writer_tools = []
if writing_assistant_tool:
    writer_tools.append(writing_assistant_tool)

editor_tools = []
if editing_assistant_tool:
    editor_tools.append(editing_assistant_tool)
if seo_optimizer_tool:
    editor_tools.append(seo_optimizer_tool)

# Content Researcher Agent
researcher = Agent(
    role="Senior Content Researcher",
    goal="Research and gather comprehensive information on given topics to support content creation",
    backstory="""You are an experienced researcher with 10+ years in digital marketing research.
    You excel at finding credible sources, identifying market trends, and extracting actionable
    insights from complex data. You have access to web search tools to gather information.""",
    tools=researcher_tools,
    verbose=True,
    allow_delegation=False
)

# Content Writer Agent
writer = Agent(
    role="Senior Content Writer",
    goal="Create engaging, high-quality content based on research and brand guidelines",
    backstory="""You are a seasoned content writer with expertise in creating compelling marketing content.
    You understand how to transform research into engaging narratives that resonate with target audiences.
    You use writing assistance tools to ensure quality.""",
    tools=writer_tools,
    verbose=True,
    allow_delegation=False
)

# Content Editor Agent
editor = Agent(
    role="Senior Content Editor",
    goal="Review, refine, and ensure content quality meets brand standards and objectives",
    backstory="""You are an experienced content editor with a keen eye for detail, brand consistency,
    and audience engagement. You excel at polishing content to perfection while maintaining the original message.
    You leverage editorial and SEO tools for comprehensive review.""",
    tools=editor_tools,
    verbose=True,
    allow_delegation=False
)

# Display agent configuration
print("Content Marketing Team Assembled!")
print("=" * 50)
for agent in [researcher, writer, editor]:
    print(f"\n{agent.role}:")
    print(f"  Goal: {agent.goal[:60]}...")
    print(f"  Tools: {len(agent.tools)} available")
    if agent.tools:
        for tool in agent.tools:
            print(f"    - {tool.name}")
    else:
        print("    - No tools (will rely on LLM capabilities)")
print("=" * 50)

## ✅ Agents Successfully Created!

### Team Composition:

1. **Senior Content Researcher**
   - Goal: Research and gather comprehensive information
   - Tools: Web search, content analysis (based on availability)
   - Delegation: Disabled

2. **Senior Content Writer**
   - Goal: Create engaging, high-quality content
   - Tools: Writing assistant, grammar checker
   - Delegation: Disabled

3. **Senior Content Editor**
   - Goal: Review and ensure content quality
   - Tools: Editing assistant, brand guidelines, SEO optimizer
   - Delegation: Disabled

**Note**: The actual number of tools available to each agent depends on successful initialization (see tool setup output above).

## 📋 Task Definition & Dependencies

CrewAI tasks are **structured work items** that define what needs to be accomplished. Tasks have:

### Core Task Attributes:
- **Description** - What needs to be done
- **Agent** - Who will execute the task
- **Expected Output** - Success criteria
- **Context** - Dependencies on other tasks
- **Tools** - Task-specific capabilities

### Task Workflow Patterns:
- **Sequential**: Tasks executed in order
- **Parallel**: Independent tasks run simultaneously
- **Conditional**: Tasks based on previous outcomes

In [ ]:
# Create tasks.yaml configuration - Shows how tools are used in tasks
tasks_yaml_content = """
research_task:
  description: Research the latest trends and best practices in AI-powered content marketing for 2024. Focus on practical applications, case studies, and emerging technologies.
  agent: content_researcher
  expected_output: A comprehensive research report with key trends, statistics, case studies, and actionable insights for AI content marketing.
  tools_used:
  - web_search         # For finding current information
  priority: high
  estimated_duration: 45 minutes

writing_task:
  description: Create an engaging blog post about AI-powered content marketing based on the research findings. The post should be informative, engaging, and targeted at marketing professionals.
  agent: content_writer
  expected_output: A well-structured 1200-1500 word blog post with introduction, key sections, examples, and conclusion. Include engaging headlines and clear takeaways.
  context:
  - research_task
  tools_used:
  - writing_assistant  # For content structure and ideas
  priority: high
  estimated_duration: 60 minutes

editing_task:
  description: Review and refine the blog post for clarity, engagement, brand consistency, and SEO optimization. Ensure the content meets quality standards and brand guidelines.
  agent: content_editor
  expected_output: A polished, publication-ready blog post with improved flow, corrected grammar, optimized headlines, and SEO enhancements.
  context:
  - research_task
  - writing_task
  tools_used:
  - editing_assistant  # For editorial improvements
  - seo_optimizer      # For search optimization
  priority: medium
  estimated_duration: 30 minutes
"""

# Parse YAML for documentation
tasks_config = yaml.safe_load(tasks_yaml_content.strip())

print("Task Configuration with Tool Usage:")
print("=" * 50)
for task_name, config in tasks_config.items():
    print(f"\n{task_name}:")
    print(f"  Agent: {config['agent']}")
    print(f"  Tools Used: {', '.join(config.get('tools_used', ['LLM only']))}")
    print(f"  Dependencies: {', '.join(config.get('context', ['None']))}")
    print(f"  Priority: {config['priority']}")
print("=" * 50)

### tasks.yaml Configuration

**YAML Configuration loaded successfully!**  
- **Tasks defined:** 3

**Task Workflow & Dependencies:**

📌 **Research Task**
- Agent: content_researcher
- Dependencies: None

📌 **Writing Task**
- Agent: content_writer
- Dependencies: research_task

📌 **Editing Task**
- Agent: content_editor
- Dependencies: research_task, writing_task

In [ ]:
# Create Task instances with dependencies

# Task 1: Research (no dependencies)
research_task = Task(
    description="""Research the latest trends and best practices in AI-powered content marketing for 2024.
    Focus on practical applications, case studies, and emerging technologies that marketing teams can implement.

    Key areas to investigate:
    - AI content generation tools and platforms
    - Personalization and audience targeting
    - Performance metrics and ROI measurement
    - Case studies of successful implementations
    - Future trends and predictions""",
    agent=researcher,
    expected_output="""A comprehensive research report containing:
    - Executive summary of key trends
    - 5-7 major trend categories with detailed analysis
    - At least 3 real-world case studies
    - Statistical data and market insights
    - Actionable recommendations for marketing teams"""
)

# Task 2: Writing (depends on research)
writing_task = Task(
    description="""Create an engaging and informative blog post about AI-powered content marketing
    based on the research findings. The content should be targeted at marketing professionals
    and business leaders looking to implement AI in their content strategies.

    Content requirements:
    - Compelling headline and introduction
    - Clear structure with subheadings
    - Include specific examples and case studies
    - Actionable tips and recommendations
    - Professional yet engaging tone
    - 1200-1500 words""",
    agent=writer,
    expected_output="""A well-structured blog post containing:
    - Attention-grabbing headline
    - Engaging introduction (100-150 words)
    - 4-6 main sections with clear subheadings
    - Real-world examples and case studies
    - Actionable takeaways for readers
    - Strong conclusion with call-to-action
    - 1200-1500 words total""",
    context=[research_task]  # This task depends on research completion
)

# Task 3: Editing (depends on both research and writing)
editing_task = Task(
    description="""Review and refine the blog post to ensure it meets high-quality standards
    for publication. Focus on clarity, engagement, brand consistency, and SEO optimization.

    Editing checklist:
    - Grammar, spelling, and punctuation accuracy
    - Flow and readability improvements
    - Brand voice and tone consistency
    - SEO optimization (keywords, meta descriptions)
    - Fact-checking and source verification
    - Call-to-action effectiveness""",
    agent=editor,
    expected_output="""A polished, publication-ready blog post with:
    - Perfect grammar and spelling
    - Improved flow and readability
    - Consistent brand voice
    - SEO-optimized headlines and structure
    - Verified facts and properly cited sources
    - Strong call-to-action
    - Editorial notes on changes made""",
    context=[research_task, writing_task]  # Depends on both previous tasks
)

print(f"\nTask Workflow:")
print(f"  1. {research_task.description.split('.')[0]}...")
print(f"     Agent: {research_task.agent.role}")
print(f"     Dependencies: None (starting task)")

print(f"\n  2. {writing_task.description.split('.')[0]}...")
print(f"     Agent: {writing_task.agent.role}")
print(f"     Dependencies: Research Task")

print(f"\n  3. {editing_task.description.split('.')[0]}...")
print(f"     Agent: {editing_task.agent.role}")
print(f"     Dependencies: Research Task + Writing Task")

## 👥 Crew Orchestration & Processes

The **Crew** is CrewAI's core orchestration component that manages how agents collaborate to complete tasks. 

### Crew Process Types:

1. **Sequential** - Tasks executed one after another (most common)
2. **Hierarchical** - Manager agent coordinates subordinates
3. **Consensual** - Agents collaborate to reach consensus

### Human Crew Abstraction:

CrewAI mimics real-world team dynamics:
- **Role Clarity**: Everyone knows their responsibilities
- **Collaboration**: Agents share context and build on each other's work
- **Delegation**: Senior roles can delegate to juniors
- **Quality Control**: Review and refinement cycles
- **Memory**: Teams remember past interactions and decisions

In [ ]:
# Create the Content Marketing Crew
content_crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[research_task, writing_task, editing_task],
    process=Process.sequential,  # Tasks executed in dependency order
    verbose=True,  # Enable detailed logging
    memory=False  # Disabled to keep demo simple (context still passed via task dependencies)
)

print("Content Marketing Crew Assembled!")
print(f"\nCrew Configuration:")
print(f"  • Team Size: {len(content_crew.agents)} agents")
print(f"  • Workload: {len(content_crew.tasks)} tasks")
print(f"  • Process: {content_crew.process.value} execution")
print(f"  • Memory: {'Enabled' if content_crew.memory else 'Disabled (using explicit context passing)'}")
print(f"  • Verbose: {'Enabled' if content_crew.verbose else 'Disabled'}")

print(f"\nTeam Hierarchy & Roles:")
for i, agent in enumerate(content_crew.agents, 1):
    delegation_status = "Can delegate" if agent.allow_delegation else "Individual contributor"
    print(f"  {i}. {agent.role}")
    print(f"     Status: {delegation_status}")
    print(f"     Focus: {agent.goal[:50]}...")

print(f"\nWorkflow Process:")
for i, task in enumerate(content_crew.tasks, 1):
    dependencies = len(task.context) if hasattr(task.context, '__len__') else 0
    print(f"  Step {i}: {task.agent.role}")
    print(f"          Dependencies: {dependencies} task(s)")
    print(f"          Output: {task.expected_output.split('.')[0]}...")

## 🚀 Complete Workflow Demo: Marketing Content Creation

Now let's see our **Content Marketing Crew** in action! This is where CrewAI's power really shines - watching specialized agents collaborate like a real marketing team.

### Workflow Overview:
1. **Research Phase**: Senior Researcher gathers market intelligence
2. **Creation Phase**: Content Writer transforms research into engaging content
3. **Refinement Phase**: Editor polishes and optimizes for publication

### Expected Collaboration Patterns:
- **Context Sharing**: Each agent builds on previous work
- **Quality Escalation**: Each step improves the output
- **Role Specialization**: Agents focus on their expertise

Let's execute our crew and observe the human-like collaboration!

In [ ]:
print("Starting Content Marketing Crew Execution...")
print("=" * 70)

result = content_crew.kickoff()

print("\nCrew Workflow Completed Successfully!")
print("=" * 70)
print("FINAL DELIVERABLE")
print("=" * 70)
print(result)
print("=" * 70)

## 📋 Summary

This notebook demonstrated **CrewAI's comprehensive approach** to multi-agent AI systems through hands-on examples.

### 🤖 **Agents** (Role-Based Team Members)
✅ **Comprehensive personas** - Role, goal, backstory, tools  
✅ **YAML configuration** - Scalable team management  
✅ **Specialization** - Domain experts with focused capabilities  
✅ **Delegation support** - Hierarchical team structures  

### 📋 **Tasks** (Structured Work Items)
✅ **Clear specifications** - Description, expected output, context  
✅ **Dependency management** - Sequential and parallel workflows  
✅ **Context sharing** - Tasks build on previous work  
✅ **Quality gates** - Review and refinement cycles  

### 👥 **Crew** (Team Orchestration)
✅ **Process types** - Sequential, hierarchical, consensual  
✅ **Memory management** - Persistent crew knowledge  
✅ **Human integration** - Human-in-the-loop workflows  
✅ **Production features** - Monitoring, error handling, scalability  

## 📚 **Resources**

- **GitHub**: [crewAI-Inc/crewAI](https://github.com/crewAI-Inc/crewAI)
- **Documentation**: [CrewAI Docs](https://docs.crewai.com/)
- **CrewAI Studio**: Visual workflow builder
- **Community**: Discord and forum support

**CrewAI represents the future of enterprise multi-agent AI - role-based, scalable, and built for real-world team collaboration! 🚀**